In [1]:
import io
import requests
import webbrowser
import mysql.connector
import tkinter as tk
import tkinter.font as tkFont
from bs4 import BeautifulSoup
from tkinter import Toplevel
from tkinter import END
from tkinter import *
from urllib.request import urlopen
from PIL import ImageTk,Image

mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Arnyroid"
)
# print(mydb)

In [2]:
conn = mydb.cursor()
# conn.execute("CREATE DATABASE IF NOT EXISTS project_ics214;")
conn.execute("USE ics214_project_wishlist;")
# conn.execute("CREATE TABLE IF NOT EXISTS login(un VARCHAR(100), pw VARCHAR(100))")
# q = "INSERT into LOGIN(un,pa) VALUES('{}','{}'))".format("admin","123456")
# conn.execute(q)
# conn.execute("CREATE TABLE IF NOT EXISTS amazon(id int AUTO_INCREMENT PRIMARY KEY, name VARCHAR(1000) NOT NULL, price varchar(200), url VARCHAR(2000));")
# conn.execute("ALTER TABLE amazon AUTO_INCREMENT = 1;")
# conn.execute("CREATE TABLE IF NOT EXISTS flipkart(id int AUTO_INCREMENT PRIMARY KEY, name VARCHAR(1000) NOT NULL, price varchar(200), url VARCHAR(2000));")
# conn.execute("ALTER TABLE flipkart AUTO_INCREMENT = 1;")
mydb.commit()

In [3]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}


In [4]:
global flipkar,amazo
flipkar=''
amazo=''

In [5]:
def show_ama() :
    q = "SELECT id, name, price from amazon;"
    conn.execute(q)
    c = conn.fetchall()
    # for i in range(len(c)) :
    #     print(c[i])
    return c

In [6]:
def show_fli() :
    q = "SELECT id, name, price from flipkart;"
    conn.execute(q)
    c = conn.fetchall()
    # for i in range(len(c)) :
    #     print(c[i])
    return c

In [7]:
def flipkart(name):
    fl_de = {}
    try:
        name1 = name.replace(" ","%20")
        flipkar=f'https://www.flipkart.com/search?q={name1}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
        res = requests.get(flipkar,headers=headers)


        # print("\nSearching in flipkart....")
        soup = BeautifulSoup(res.text,'html.parser')
        
        if(soup.select('._4rR01T')):
            flipkart_name = soup.select('._4rR01T')[0].getText().strip().upper()
            print(flipkart_name)
            name = name.upper()
            nameL = name.split()
            print(nameL)
            leng = len(nameL)
            print(leng)
            count = 0
            for k in nameL :
                if k in flipkart_name :
                    count+=1
            print(count)
            if(count == leng) :
                flipkart_price = soup.select('._30jeq3')[0].getText().strip()
                flipkart_name = soup.select('._4rR01T')[0].getText().strip()
                # print("Flipkart:")
                # print(flipkart_name)
                # print(flipkart_price)
                # print("---------------------------------")
                flipkart_price = flipkart_price.replace("₹","")
                fl_de[0] = flipkart_price
                fl_de[1] = flipkart_name
                fl_de[2] = flipkar
                
        elif(soup.select('.s1Q9rs')):
            flipkart_name = soup.select('.s1Q9rs')[0].getText().strip().upper()
            # print(flipkart_name)
            name = name.upper()
            nameL = name.split()
            # print(nameL)
            leng = len(nameL)
            # print(leng)
            count = 0
            for k in nameL :
                if k in flipkart_name :
                    count+=1
                    # print(count)
            flipkart_name = soup.select('.s1Q9rs')[0].getText().strip()
            flipkart_name = flipkart_name.upper()
            if (count == leng):
                flipkart_price = soup.select('._30jeq3')[0].getText().strip()
                flipkart_name = soup.select('.s1Q9rs')[0].getText().strip()
                # print("Flipkart:")
                # print(flipkart_name)
                # print(flipkart_price)
                # print("---------------------------------")
                fl_de[0] = flipkart_price
                fl_de[1] = flipkart_name
                fl_de[2] = flipkar
        else:
            flipkart_price='0'
            
        return fl_de 
    except:
        # print("Flipkart: No product found!")  
        # print("---------------------------------")
        flipkart_price= '0'
    return fl_de

def amazon(name):
    am_de = {}
    try:
        global amazon
        name1 = name.replace(" ","-")
        name2 = name.replace(" ","+")
        # amazo=f'https://www.amazon.in/{name1}/s?k={name2}'
        amazo = "https://www.amazon.in/s?k={}".format(name2)
        res = requests.get(amazo,headers=headers)
        # print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text,'html.parser')
        # print(soup)
        amazon_page = soup.select('.a-color-base.a-text-normal')
        # print(amazon_page)
        amazon_page_length = int(len(amazon_page))
        for i in range(0,amazon_page_length):
            name = name.upper()
            amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
            nameL = name.split()
            print(nameL)
            leng = len(nameL)
            print(leng)
            count = 0
            for k in nameL :
                if k in amazon_name :
                    count+=1
                    print(count)
            if(count == leng) :
                amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip()
                amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                # print("Amazon:")
                # print(amazon_name)
                # print("₹"+amazon_price)
                # print("---------------------------------") 
                am_de[0] = amazon_price
                am_de[1] = amazon_name
                am_de[2] = amazo
                break
            else:
                i+=1
                i=int(i)
                if i==amazon_page_length:
                    amazon_price = '0'
                    # print("amazon : No product found!")
                    # print("-----------------------------")
                    break
                    
        return am_de
    except:
        # print("Amazon: No product found!")
        # print("---------------------------------")
        amazon_price = '0'
    return am_de

In [8]:
global l1, l2 , flipkart_details, amazon_details
l1 = show_ama()
l2 = show_fli()

In [9]:
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Wishlist")
    width=1080
    height=720
    screenwidth = root.winfo_screenwidth()
    screenheight = root.winfo_screenheight()
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
    root.geometry(alignstr)
    root.resizable(width=False, height=False)
    root.configure(bg='#000000')
    root.iconbitmap('icon.ico')
    pic_source = 'assets/bl_bg.jpg'
    picture = Image.open(pic_source).resize((1080,720))
    pic = ImageTk.PhotoImage(picture)
    lab = Label(root, image=pic)
    lab.place(x=0,y=0,relheight=1,relwidth=1)
    
    
    
    # ADD TO LIST WINDOW
    def GButton_477_command():
        newWin = Toplevel(root)
        newWin.title("ADD TO LIST!!!")
        width=1080
        height=720
        screenwidth = newWin.winfo_screenwidth()
        screenheight = newWin.winfo_screenheight()
        alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
        newWin.iconbitmap('icon.ico')
        newWin.geometry(alignstr)
        newWin.resizable(width=False, height=False)
        newWin.configure(bg='#000000')
        # pic_source = 'assets/27230.jpg'
        # picture = Image.open(pic_source)
        # pic = ImageTk.PhotoImage(picture)
        # lab_e = Label(newWin, image=pic)
        # lab_e.place(x=0,y=0,relheight=1,relwidth=1) 
        
        def GButton_686_command():
            GListBox_252.delete(0,END)
            GListBox_956.delete(0,END)
            search_query = GLineEdit_224.get()
            flipkart_details=flipkart(search_query)
            amazon_details=amazon(search_query)
            for i in range (len(flipkart_details) - 2, -1, -1) :
                GListBox_252.insert(i,flipkart_details[i])
            for i in range (len(amazon_details) - 2, -1, -1) :
                GListBox_956.insert(i,amazon_details[i])
        
            def GButton_412_command() :
                webbrowser.open(amazon_details[2])
                return 0
            
            def GButton_750_command() :
                webbrowser.open(flipkart_details[2])
                return 0
            
            def GButton_255_command():
                insert = "INSERT INTO amazon(name,price,url) VALUES('{}','{}','{}')".format(amazon_details[1],amazon_details[0],amazon_details[2])
                conn.execute(insert)
                mydb.commit()
                insert = "INSERT INTO flipkart(name,price,url) VALUES('{}','{}','{}')".format(flipkart_details[1],flipkart_details[0],flipkart_details[2])
                conn.execute(insert)
                mydb.commit()
                # print("command")
            
            def on_enter_412(e) :
                GButton_412["fg"] = "#000000"
                GButton_412["bg"] = "#f0f0f0"
            def on_leave_412(e) :
                GButton_412["fg"] = "#f0f0f0"
                GButton_412["bg"] = "#000000"
            
            GButton_412=tk.Button(newWin)
            GButton_412["fg"] = "#f0f0f0"
            ft = tkFont.Font(family='Times',size=10)
            GButton_412["font"] = ft
            GButton_412["bg"] = "#000000"
            GButton_412["justify"] = "center"
            GButton_412["text"] = "Visit"
            GButton_412.place(x=940,y=500,width=50,height=98)
            GButton_412["command"] = GButton_412_command

            GButton_412.bind("<Enter>", on_enter_412)
            GButton_412.bind("<Leave>", on_leave_412)

            def on_enter_750(e) :
                GButton_750["fg"] = "#000000"
                GButton_750["bg"] = "#f0f0f0"
            def on_leave_750(e) :
                GButton_750["fg"] = "#f0f0f0"
                GButton_750["bg"] = "#000000"
            
            GButton_750=tk.Button(newWin)
            GButton_750["fg"] = "#f0f0f0"
            ft = tkFont.Font(family='Times',size=10)
            GButton_750["font"] = ft
            GButton_750["bg"] = "#000000"
            GButton_750["justify"] = "center"
            GButton_750["text"] = "Visit"
            GButton_750.place(x=940,y=310,width=50,height=98)
            GButton_750["command"] = GButton_750_command

            GButton_750.bind("<Enter>", on_enter_750)
            GButton_750.bind("<Leave>", on_leave_750)
            
            def on_enter_255(e) :
                GButton_255["fg"] = "#000000"
                GButton_255["bg"] = "#f0f0f0"
            def on_leave_255(e) :
                GButton_255["fg"] = "#f0f0f0"
                GButton_255["bg"] = "#000000" 
            
            
            GButton_255=tk.Button(newWin)
            GButton_255["fg"] = "#f0f0f0"
            ft = tkFont.Font(family='Times',size=10)
            GButton_255["font"] = ft
            GButton_255["bg"] = "#000000"
            GButton_255["justify"] = "center"
            GButton_255["text"] = "ADD"
            GButton_255.place(x=250,y=620,width=195,height=55)
            GButton_255["command"] = GButton_255_command
            
            GButton_255.bind("<Enter>",on_enter_255)
            GButton_255.bind("<Leave>",on_leave_255)
            
            # print("command")

        def GButton_888_command():
            GLineEdit_224.delete(0,END)
            GListBox_252.delete(0,END)
            GListBox_956.delete(0,END)
            # print("command")
            
        
        def on_enter_888(e) :
            GButton_888["fg"] = "#000000"
            GButton_888["bg"] = "#f0f0f0"
        def on_leave_888(e) :
            GButton_888["fg"] = "#f0f0f0"
            GButton_888["bg"] = "#000000"    
        
        GButton_888=tk.Button(newWin)
        GButton_888["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=10)
        GButton_888["font"] = ft
        GButton_888["bg"] = "#000000"
        GButton_888["justify"] = "center"
        GButton_888["text"] = "Clear"
        GButton_888.place(x=840,y=170,width=150,height=51)
        GButton_888["command"] = GButton_888_command
        
        GButton_888.bind("<Enter>",on_enter_888)
        GButton_888.bind("<Leave>",on_leave_888)

        GLineEdit_224=tk.Entry(newWin)
        GLineEdit_224["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=24)
        GLineEdit_224["font"] = ft
        GLineEdit_224["fg"] = "#f0f0f0"
        GLineEdit_224["bg"] = "#000000"
        GLineEdit_224["justify"] = "center"
        GLineEdit_224["text"] = "Entry"
        GLineEdit_224.place(x=180,y=170,width=458,height=52)

        def on_enter_686(e) :
            GButton_686["fg"] = "#000000"
            GButton_686["bg"] = "#f0f0f0"
        def on_leave_686(e) :
            GButton_686["fg"] = "#f0f0f0"
            GButton_686["bg"] = "#000000"
        
        GButton_686=tk.Button(newWin)
        GButton_686["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=10)
        GButton_686["font"] = ft
        GButton_686["bg"] = "#000000"
        GButton_686["justify"] = "center"
        GButton_686["text"] = "Search"
        GButton_686.place(x=670,y=170,width=150,height=51)
        GButton_686["command"] = GButton_686_command

        GButton_686.bind("<Enter>", on_enter_686)
        GButton_686.bind("<Leave>", on_leave_686)
        
        # FLIPKART
        GListBox_252=tk.Listbox(newWin)
        GListBox_252["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        GListBox_252["font"] = ft
        GListBox_252["bg"] = "#000000"
        GListBox_252["fg"] = "#f0f0f0"
        GListBox_252["justify"] = "center"
        GListBox_252["selectmode"] = "MULTIPLE"
        GListBox_252.place(x=180,y=310,width=737,height=98)

        # AMAZON
        GListBox_956=tk.Listbox(newWin)
        GListBox_956["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        GListBox_956["font"] = ft
        GListBox_956["bg"] = "#000000"
        GListBox_956["fg"] = "#f0f0f0"
        GListBox_956["justify"] = "center"
        GListBox_956["selectmode"] = "MULTIPLE"
        GListBox_956.place(x=180,y=500,width=737,height=98)
        
        GLabel_753=tk.Label(newWin)
        ft = tkFont.Font(family='Times',size=34)
        GLabel_753["font"] = ft
        GLabel_753["bg"] = "#000000"
        GLabel_753["fg"] = "#f0f0f0"
        GLabel_753["justify"] = "center"
        GLabel_753["text"] = "SEARCH AND ADD"
        GLabel_753.place(x=360,y=50,width=400,height=64)
        
        GLabel_167=tk.Label(newWin)
        ft = tkFont.Font(family='Times',size=15)
        GLabel_167["font"] = ft
        GLabel_167["bg"] = "#000000"
        GLabel_167["fg"] = "#f0f0f0"
        GLabel_167["justify"] = "center"
        GLabel_167["text"] = "Product on flipkart"
        GLabel_167.place(x=440,y=260,width=224,height=30)

        GLabel_239=tk.Label(newWin)
        ft = tkFont.Font(family='Times',size=15)
        GLabel_239["font"] = ft
        GLabel_239["bg"] = "#000000"
        GLabel_239["fg"] = "#f0f0f0"
        GLabel_239["justify"] = "center"
        GLabel_239["text"] = "Product on amazon"
        GLabel_239.place(x=440,y=450,width=222,height=31)
        
        def on_enter_951(e) :
            GButton_951["fg"] = "#000000"
            GButton_951["bg"] = "#f0f0f0"
        def on_leave_951(e) :
            GButton_951["fg"] = "#f0f0f0"
            GButton_951["bg"] = "#000000"    
        
        GButton_951=tk.Button(newWin)
        GButton_951["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=10)
        GButton_951["font"] = ft
        GButton_951["bg"] = "#000000"
        GButton_951["justify"] = "center"
        GButton_951["text"] = "Back"
        GButton_951.place(x=650,y=620,width=195,height=55)
        GButton_951["command"] = newWin.destroy

        GButton_951.bind("<Enter>", on_enter_951)
        GButton_951.bind("<Leave>", on_leave_951)
                
        # print("command")

    # DELETE WINDOW
    def GButton_544_command():
        del_win = Toplevel(root)
        width=1080
        height=720
        screenwidth = del_win.winfo_screenwidth()
        screenheight = del_win.winfo_screenheight()
        alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
        del_win.geometry(alignstr)
        del_win.resizable(width=False, height=False)
        del_win.configure(bg='#000000')
        del_win.iconbitmap('assets/icon1.ico')
        # pic_source = 'assets/27230.jpg'
        # picture = Image.open(pic_source).resize((1080,720))
        # pic = ImageTk.PhotoImage(picture)
        # lab_e = Label(del_win, image=pic)
        # lab_e.place(x=0,y=0,relheight=1,relwidth=1)
        
        def GButton_655_command():
            ind = GLineEdit_766.get()
            q = "DELETE from amazon where id = {}".format(ind)
            conn.execute(q)
            mydb.commit()
            q = "DELETE from flipkart where id = {}".format(ind)
            conn.execute(q)
            mydb.commit()
            q = "Select count(*) from amazon;"
            conn.execute(q)
            count = conn.fetchall()
            p = int(count[0][0])
            print(p)
            flag=int(ind)
            for l in range (int(ind)+1,1000) :
                try :
                    que = "UPDATE amazon SET id={} where id={};".format(str(flag),str(l))
                    conn.execute(que)
                    mydb.commit()
                    que = "UPDATE flipkart SET id={} where id={};".format(str(flag),str(l))
                    conn.execute(que)
                    mydb.commit()
                    flag=flag+1
                except:
                    pass
            
            GLabel_357=tk.Label(del_win)
            ft = tkFont.Font(family='Times',size=20)
            GLabel_357["font"] = ft
            GLabel_357["fg"] = "#f0f0f0"
            GLabel_357["bg"] = "#000000"
            GLabel_357["justify"] = "center"
            GLabel_357["text"] = "Item removed"
            GLabel_357.place(x=420,y=260,width=195,height=55)
            
            GLineEdit_766.delete(0,END)
            # print("command")
        
        GLabel_750=tk.Label(del_win)
        ft = tkFont.Font(family='Times',size=34)
        GLabel_750["font"] = ft
        GLabel_750["fg"] = "#f0f0f0"
        GLabel_750["bg"] = "#000000"
        GLabel_750["justify"] = "center"
        GLabel_750["text"] = "REMOVE ITEM"
        GLabel_750.place(x=340,y=50,width=400,height=64)
        
        enter_id_label=tk.Label(del_win)
        ft = tkFont.Font(family='Times',size=20)
        enter_id_label["font"] = ft
        enter_id_label["fg"] = "#f0f0f0"
        enter_id_label["bg"] = "#000000"
        enter_id_label["justify"] = "center"
        enter_id_label["text"] = "Enter ID :"
        enter_id_label.place(x=180,y=170,width=200,height=52)

        GLineEdit_766=tk.Entry(del_win)
        GLineEdit_766["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=20)
        GLineEdit_766["font"] = ft
        GLineEdit_766["bg"] = "#000000"
        GLineEdit_766["fg"] = "#f0f0f0"
        GLineEdit_766["justify"] = "center"
        GLineEdit_766.place(x=380,y=170,width=258,height=52)
        
        def on_enter_655(e) :
            GButton_655["fg"] = "#000000"
            GButton_655["bg"] = "#f0f0f0"
        def on_leave_655(e) :
            GButton_655["fg"] = "#f0f0f0"
            GButton_655["bg"] = "#000000"    

        GButton_655=tk.Button(del_win)
        GButton_655["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_655["font"] = ft
        GButton_655["bg"] = "#000000"
        GButton_655["justify"] = "center"
        GButton_655["text"] = "Confirm"
        GButton_655.place(x=670,y=170,width=150,height=51)
        GButton_655["command"] = GButton_655_command

        GButton_655.bind("<Enter>", on_enter_655)
        GButton_655.bind("<Leave>", on_leave_655)
        
        def on_enter_628(e) :
            GButton_628["fg"] = "#000000"
            GButton_628["bg"] = "#f0f0f0"
        def on_leave_628(e) :
            GButton_628["fg"] = "#f0f0f0"
            GButton_628["bg"] = "#000000"    
        
        GButton_628=tk.Button(del_win)
        GButton_628["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_628["font"] = ft
        GButton_628["bg"] = "#000000"
        GButton_628["justify"] = "center"
        GButton_628["text"] = "Back"
        GButton_628.place(x=420,y=360,width=195,height=55)
        GButton_628["command"] = del_win.destroy

        GButton_628.bind("<Enter>", on_enter_628)
        GButton_628.bind("<Leave>", on_leave_628)
        
        # print("command")

    def GButton_992_command() :        
        def discover():
            class Discover:
                def __init__(self):
                    self.d = requests.get('https://newsapi.org/v2/top-headlines?country=in&category=technology&apiKey=ec6ac7653da24feb8709ce7b886253f7').json()
                    
                    self.gui()
                    
                    self.load_discover(0)

                def gui(self):
                    self.discover_win = Toplevel(root)
                    self.discover_win.geometry('1080x720')
                    self.discover_win.resizable(0,0)
                    self.discover_win.title('Discover TECHNOLOGY....')
                    self.discover_win.configure(background='black')

                def clear(self):
                    for i in self.discover_win.pack_slaves():
                        i.destroy()

                def load_discover(self,index):

                    self.clear()
                    try:
                        pic_source = self.d['articles'][index]['urlToImage']
                        my_d = urlopen(pic_source).read()
                        picture = Image.open(io.BytesIO(my_d)).resize((550,300))
                        pic = ImageTk.PhotoImage(picture)
                    except:
                        pic_source = 'assets/960x0.jpg'
                        # my_d = urlopen(pic_source).read()
                        picture = Image.open(pic_source).resize((550, 400))
                        pic = ImageTk.PhotoImage(picture)


                    label = Label(self.discover_win,image=pic)
                    label.pack()

                    # pic_source = 'assets/bl_bg.jpg'
                    # picture = Image.open(pic_source).resize((1080,720))
                    # pic = ImageTk.PhotoImage(picture)
                    # lab_e = Label(self.discover_win, image=pic)
                    # lab_e.pack()

                    my_title = Label(self.discover_win,text=self.d['articles'][index]['title'],bg='black',fg='white',wraplength=450,justify='center')
                    my_title.pack(pady=(10,20))
                    my_title.config(font=('times',15))

                    describe = Label(self.discover_win, text=self.d['articles'][index]['description'], bg='black', fg='white', wraplength=650,justify='center')
                    describe.pack(pady=(2, 20))
                    describe.config(font=('times', 12))

                    frame = Frame(self.discover_win,bg='black')
                    frame.pack(expand=True,fill=BOTH)

                    if index != 0:
                        def on_leave_prev(e) :
                            prev["bg"] = "#000000"
                            prev["fg"] = "#f0f0f0"
                        def on_entry_prev(e) :
                            prev["bg"] = "#f0f0f0"
                            prev["fg"] = "#000000"
                        prev = Button(frame,text='Prev',width=16,height=3,command=lambda :self.load_discover(index-1))
                        prev.pack(padx = 125,side=LEFT)
                        
                        prev.bind("<Enter>",on_entry_prev)
                        prev.bind("<Leave>",on_leave_prev)

                    def on_leave_read(e) :
                        read["bg"] = "#000000"
                        read["fg"] = "#f0f0f0"
                    def on_entry_read(e) :
                        read["bg"] = "#f0f0f0"
                        read["fg"] = "#000000"
                    read = Button(frame, text='Read More', width=16, height=3,command=lambda :self.open_link(self.d['articles'][index]['url']))
                    read.pack(padx=100,side=LEFT)
                    
                    read.bind("<Enter>",on_entry_read)
                    read.bind("<Leave>",on_leave_read)

                    if index != len(self.d['articles'])-1:
                        
                        def on_leave_next(e) :
                            next["bg"] = "#000000"
                            next["fg"] = "#f0f0f0"
                        def on_entry_next(e) :
                            next["bg"] = "#f0f0f0"
                            next["fg"] = "#000000"
                        
                        next = Button(frame, text='Next', width=16, height=3,command=lambda :self.load_discover(index+1))
                        next.pack(padx=125,side=LEFT)
                        
                        next.bind("<Enter>",on_entry_next)
                        next.bind("<Leave>",on_leave_next)

                    self.discover_win.mainloop()

                def open_link(self,url):
                    webbrowser.open(url)
            
            obj = Discover()
        discover()
        # print("command")
    
    def welcome_page():
        GLabel_851.destroy()
        GButton_823.destroy()
        GLabel_7.destroy()
        GLabel_712.destroy()
        GLineEdit_614.destroy()
        GLabel_8.destroy()
        GLineEdit_6.destroy()
        
        def GButton_748_command():
            Glabel_722.delete(0,END)
            GLabel_192.delete(0,END)
            l1 = show_ama()
            l2 = show_fli()
            for i in range(len(l1)) :
                Glabel_722.insert(i,l1[i])
            for i in range(len(l2)) :
                GLabel_192.insert(i,l2[i])
            
            # print("command")
        GLabel_983=tk.Label(root)
        GLabel_983["anchor"] = "center"
        GLabel_983["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=32)
        GLabel_983["font"] = ft
        GLabel_983["bg"] = "#000000"
        GLabel_983["justify"] = "center"
        GLabel_983["text"] = "Welcome to your WISHLIST"
        GLabel_983["relief"] = "raised"
        GLabel_983.place(x=270,y=60,width=550,height=50)

        def on_enter_477(e) :
            GButton_477["fg"] = "#000000"
            GButton_477["bg"] = "#f0f0f0"
        def on_leave_477(e) :
            GButton_477["fg"] = "#f0f0f0"
            GButton_477["bg"] = "#000000"
        
        GButton_477=tk.Button(root)
        GButton_477["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_477["font"] = ft
        GButton_477["bg"] = "#000000"
        GButton_477["justify"] = "center"
        GButton_477["text"] = "Search"
        GButton_477["relief"] = "raised"
        GButton_477.place(x=240,y=150,width=100,height=40)
        GButton_477["command"] = GButton_477_command
        
        GButton_477.bind("<Enter>", on_enter_477)
        GButton_477.bind("<Leave>", on_leave_477)
        
        ft = tkFont.Font(family='Times',size=10)
        Glabel_722 = tk.Listbox(root, bg = "#000000",fg="#f0f0f0", font = ft,xscrollcommand=True)
        Glabel_722.place(x=40,y=310,width=483,height=274)
        
        ft = tkFont.Font(family='Times',size=10)
        GLabel_192 = tk.Listbox(root,bg = "#000000",fg="#f0f0f0",font = ft,xscrollcommand=True)
        GLabel_192.place(x=560,y=310,width=483,height=274)
        
        def on_enter(e) :
            GButton_748["fg"] = "#000000"
            GButton_748["bg"] = "#f0f0f0"
        def on_leave(e) :
            GButton_748["fg"] = "#f0f0f0"
            GButton_748["bg"] = "#000000"
            

        GButton_748=tk.Button(root)
        GButton_748["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_748["font"] = ft
        GButton_748["bg"] = "#000000"
        GButton_748["justify"] = "center"
        GButton_748["text"] = "Refresh"
        GButton_748.place(x=750,y=150,width=100,height=40)
        GButton_748["command"] = GButton_748_command
        
        GButton_748.bind("<Enter>", on_enter)
        GButton_748.bind("<Leave>", on_leave)
        
        def invisible_button_command() :
            Glabel_722.delete(0,END)
            GLabel_192.delete(0,END)
            l1 = show_ama()
            l2 = show_fli()
            for i in range(len(l1)) :
                Glabel_722.insert(i,l1[i])
            for i in range(len(l2)) :
                GLabel_192.insert(i,l2[i])
        
        invisible_button=tk.Button(root)
        invisible_button["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=10)
        invisible_button["font"] = ft
        invisible_button["bg"] = "#000000"
        invisible_button["justify"] = "center"
        invisible_button["text"] = ""
        invisible_button.place(x=1,y=1,width=1,height=1)
        invisible_button["command"] = invisible_button_command()

        GLabel_340=tk.Label(root)
        ft = tkFont.Font(family='Times',size=20)
        GLabel_340["font"] = ft
        GLabel_340["bg"] = "#000000"
        GLabel_340["fg"] = "#f0f0f0"
        GLabel_340["justify"] = "center"
        GLabel_340["text"] = "AMAZON"
        GLabel_340.place(x=200,y=260,width=150,height=35)

        GLabel_269=tk.Label(root)
        ft = tkFont.Font(family='Times',size=20)
        GLabel_269["font"] = ft
        GLabel_269["bg"] = "#000000"
        GLabel_269["fg"] = "#f0f0f0"
        GLabel_269["justify"] = "center"
        GLabel_269["text"] = "FLIPKART"
        GLabel_269.place(x=730,y=260,width=150,height=35)
        
        def on_enter_544(e) :
            GButton_544["fg"] = "#000000"
            GButton_544["bg"] = "#f0f0f0"
        def on_leave_544(e) :
            GButton_544["fg"] = "#f0f0f0"
            GButton_544["bg"] = "#000000"    
        
        GButton_544=tk.Button(root)
        GButton_544["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_544["font"] = ft
        GButton_544["bg"] = "#000000"
        GButton_544["justify"] = "center"
        GButton_544["text"] = "Delete"
        GButton_544.place(x=492,y=150,width=100,height=40)
        GButton_544["command"] = GButton_544_command

        GButton_544.bind("<Enter>", on_enter_544)
        GButton_544.bind("<Leave>", on_leave_544)
        
        def on_enter_992(e) :
            GButton_992["fg"] = "#000000"
            GButton_992["bg"] = "#f0f0f0"
        def on_leave_992(e) :
            GButton_992["fg"] = "#f0f0f0"
            GButton_992["bg"] = "#000000"    

        GButton_992=tk.Button(root)
        GButton_992["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=15)
        GButton_992["font"] = ft
        GButton_992["bg"] = "#000000"
        GButton_992["justify"] = "center"
        GButton_992["text"] = "Discover Latest in TECH"
        GButton_992.place(x=400,y=215,width=290,height=35)
        GButton_992["command"] = GButton_992_command

        GButton_992.bind("<Enter>", on_enter_992)
        GButton_992.bind("<Leave>", on_leave_992)
        
        def on_enter_899(e) :
            GButton_899["fg"] = "#000000"
            GButton_899["bg"] = "#f0f0f0"
        def on_leave_899(e) :
            GButton_899["fg"] = "#f0f0f0"
            GButton_899["bg"] = "#000000"    
        
        GButton_899=tk.Button(root)
        GButton_899["fg"] = "#f0f0f0"
        ft = tkFont.Font(family='Times',size=20)
        GButton_899["font"] = ft
        GButton_899["bg"] = "#000000"
        GButton_899["justify"] = "center"
        GButton_899["text"] = "EXIT"
        GButton_899.place(x=492,y=620,width=100,height=40)
        GButton_899["command"] = root.destroy

        GButton_899.bind("<Enter>", on_enter_899)
        GButton_899.bind("<Leave>", on_leave_899)
        
    def login_check() :
        u = GLineEdit_614.get()
        p = GLineEdit_6.get()
        q = "SELECT * FROM login;"
        conn.execute(q)
        details = conn.fetchall()
        user = details[0][0]
        passd = details[0][1]
        if(u == user and p == passd) :
            welcome_page()
        else :
            GLabel_7["anchor"] = "center"
            GLabel_7["fg"] = "#f0f0f0"
            ft = tkFont.Font(family='Times',size=20)
            GLabel_7["font"] = ft
            GLabel_7["bg"] = "#000000"
            GLabel_7["justify"] = "center"
            GLabel_7["text"] = "INCORRECT"
            GLabel_7["relief"] = "raised"
            GLabel_7.place(x=460,y=520,width=170,height=40)
        
    GLabel_851=tk.Label(root)
    GLabel_851["anchor"] = "center"
    GLabel_851["fg"] = "#f0f0f0"
    ft = tkFont.Font(family='Times',size=32)
    GLabel_851["font"] = ft
    GLabel_851["bg"] = "#000000"
    GLabel_851["justify"] = "center"
    GLabel_851["text"] = "Login"
    GLabel_851["relief"] = "raised"
    GLabel_851.place(x=270,y=60,width=550,height=50)
        
    GLabel_7=tk.Label(root)
    
    GLabel_712=tk.Label(root)
    GLabel_712["anchor"] = "center"
    GLabel_712["fg"] = "#f0f0f0"
    ft = tkFont.Font(family='Times',size=20)
    GLabel_712["font"] = ft
    GLabel_712["bg"] = "#000000"
    GLabel_712["justify"] = "center"
    GLabel_712["text"] = "Username"
    GLabel_712["relief"] = "raised"
    GLabel_712.place(x=290,y=205,width=180,height=30)
    
    GLineEdit_614=tk.Entry(root)
    GLineEdit_614["borderwidth"] = "1px"
    ft = tkFont.Font(family='Times',size=20)
    GLineEdit_614["font"] = ft
    GLineEdit_614["bg"] = "#000000"
    GLineEdit_614["fg"] = "#f0f0f0"
    GLineEdit_614["justify"] = "center"
    GLineEdit_614.place(x=290,y=250,width=180,height=52)
    
    GLabel_8=tk.Label(root)
    GLabel_8["anchor"] = "center"
    GLabel_8["fg"] = "#f0f0f0"
    ft = tkFont.Font(family='Times',size=20)
    GLabel_8["font"] = ft
    GLabel_8["bg"] = "#000000"
    GLabel_8["justify"] = "center"
    GLabel_8["text"] = "Password"
    GLabel_8["relief"] = "raised"
    GLabel_8.place(x=590,y=205,width=180,height=30)
    
    GLineEdit_6=tk.Entry(root,show="*")
    GLineEdit_6["borderwidth"] = "1px"
    ft = tkFont.Font(family='Times',size=20)
    GLineEdit_6["font"] = ft
    GLineEdit_6["bg"] = "#000000"
    GLineEdit_6["fg"] = "#f0f0f0"
    GLineEdit_6["justify"] = "center"
    GLineEdit_6.place(x=590,y=250,width=180,height=52)
    
    def on_enter_823(e) :
        GButton_823["fg"] = "#000000"
        GButton_823["bg"] = "#f0f0f0"
    def on_leave_823(e) :
        GButton_823["fg"] = "#f0f0f0"
        GButton_823["bg"] = "#000000"    
    
    GButton_823=tk.Button(root)
    GButton_823["fg"] = "#f0f0f0"
    ft = tkFont.Font(family='Times',size=20)
    GButton_823["font"] = ft
    GButton_823["bg"] = "#000000"
    GButton_823["justify"] = "center"
    GButton_823["text"] = "PROCEED"
    GButton_823.place(x=460,y=420,width=150,height=40)
    GButton_823["command"] = login_check

    GButton_823.bind("<Enter>", on_enter_823)
    GButton_823.bind("<Leave>", on_leave_823)
    
    root.mainloop()


['NORD']
1
['NORD']
1
1
SAMSUNG GALAXY S20 FE 5G (CLOUD MINT, 128 GB)
['SAMSUNG', 'S20']
2
2
['SAMSUNG', 'S20']
2
['SAMSUNG', 'S20']
2
1
2
7
